In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)
block_size = 64
batch_size = 128 #how many blocks in parallel
learning_rate = 3e-4
max_iters = 1000
eval_iters = 250
dropout = 0.2 #dropout random neurons of network = don't overfit
n_embd = 384
n_layer = 4
n_head = 4

cuda


In [2]:
chars = ""
with open('wizard-of-oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
    chars = sorted(list(set(text)))
    
vocab_size = len(chars)
chars

['\n',
 ' ',
 '!',
 '"',
 '&',
 "'",
 '(',
 ')',
 '*',
 ',',
 '-',
 '.',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 '[',
 ']',
 '_',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '\ufeff']

In [3]:
string_to_int = { ch:i for i, ch in enumerate(chars) }
int_to_string = { i:ch for i, ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s];
decode = lambda l: "".join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)

print(data[:100])

tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,
         0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1,
        36, 25, 38, 28,  1, 39, 30,  1, 39, 50])


In [4]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size, ))
    #print(ix)
    x = torch.stack([data[i: i+block_size] for i in ix])
    y = torch.stack([data[i + 1:i + block_size + 1] for i in ix])
    x, y = x.to(device), y.to(device) #pushed to GPU = will be handled there
    return x,y

x, y = get_batch('train')
print("inputs: ")
print(x)

print ("targets: ")
print(y)

inputs: 
tensor([[62, 67,  1,  ..., 58,  1, 69],
        [67, 72,  1,  ..., 62, 60, 61],
        [ 1, 73, 68,  ..., 74, 67, 73],
        ...,
        [67,  1, 73,  ..., 58,  1, 68],
        [54, 67, 72,  ..., 11,  0, 47],
        [72, 73, 71,  ...,  1, 55, 68]], device='cuda:0')
targets: 
tensor([[67,  1, 68,  ...,  1, 69, 58],
        [72,  1, 73,  ..., 60, 61, 73],
        [73, 68,  1,  ..., 67, 73, 62],
        ...,
        [ 1, 73, 61,  ...,  1, 68, 73],
        [67, 72, 76,  ...,  0, 47, 62],
        [73, 71, 54,  ..., 55, 68, 76]], device='cuda:0')


In [5]:
#decorator makes pytorch not use gradients at all in it
# = reduces computation / memory usage
#NO TRAINING IS HAPPENING HERE
@torch.no_grad()
def estimate_loss():
    out = {}

    # puts model into eval mode - for evaluating / testing
    # ** stuff disabled
    # want to see how well model actually performs
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
        
        # puts model into training mode
        # ** weights + biases update during this phase
        # ** dropout + batch normalisation active
        model.train()
    return out

In [6]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size))) #save 'no look ahead masking' to actual model state = reduce overhead

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # input of size (batch, time-step, channels)
        # output of size (batch, time-step, head size)

        B, T, C = x.shape
        k = self.key(x) # (B, T, hs)
        q = self.query(x) # (B, T, hs)

        #compute attention scores ("affinities")
        wei = q @ k.transpose(-2, -1) * k.shape[-1]**-0.5 #(B, T, hs) @ (B, hs, T) -> (B. T. T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)

        #perform the weighted aggregation of the values
        v = self.value(x) # (B, T, hs)
        out = wei @ v # (B, T, T) @ (B, T, hs) -> (B, T, hs)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)]) # heads in parallel
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1) # (B, T, F) -> (B, T, [h1, h1, h1, h2, h2, h2 ... ])
        out = self.dropout(self.proj(out))
        return out

class FeedForward(nn.Module):
    """ a simple linear layer fllowed by a non-linearity"""

    def __init__(self, n_embd):
        super().__init__()
        self.next = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout), #makes certain percentage of neurons 0 = prevents overfitting
        )

    def forward(self, x):
        return self.next(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head #how many features captured by each head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd) #linear, relu, linear
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        y = self.sa(x)
        x = self.ln1(x + y) #add and norm (residual connection)
        y = self.ffwd(x)
        x = self.ln2(x + y) #add and norm
        return x

class GPTLanuageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd) #so don't lose info about positioning of tokens
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)]) #how many sequential decoder blocks / layers

        self.ln_f = nn.LayerNorm(n_embd) # Final layer norm, helps to converge better
        self.lm_head = nn.Linear(n_embd, vocab_size) # Projecting from n_embd to vocab_size (for use with softmax)

        self.apply(self._init_weights)

    # initialise weights around standard deviation
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
    
    def forward(self, index, targets=None):

        B, T = index.shape
        
        #idx and targets are both (B, T) tensors of integers
        tok_emb = self.token_embedding_table(index) #(B, T, C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) #(T, C), torch.arange is indices from 0 to length(T) - 1
        x = tok_emb + pos_emb # (B, T, C)
        x = self.blocks(x) # (B, T, C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) #(B, T, vocab_size), to put in softmax
        
        if targets is None:
            loss = None
        else:
            # batch size, ('time steps' or 'sequence length'), 'channels' = dimensionality of vectors =  vocab_size  
            B, T, C = logits.shape
            
            #flatten, now where the characters are in their individual sequences doesn't matter
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)

            #compares the vectors from embedding table to the truth value (truth vectors automatically generated, pytorch expects only the actual truth val here)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
        
    #we're not training here, just using existing model
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            
            # focus only on the last time step AS ONLY WANT NEXT CHARACTER
            logits = logits[:, -1, :] # becomes (B, C)
            
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = GPTLanuageModel(vocab_size)
m = model.to(device) #push to GPU

In [7]:
#pytorch optimizer:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")
    
    #sample a batch of data
    xb, yb = get_batch('train') #xb = index / inputs, yb = target

    #evaluate the loss
    logits, loss = model.forward(xb, yb)

    #so previous gradients don't affect current gradient
    #previous gradients are from previous data
    #none instead of 0 as optimisation for less space (0 would be stored as int64)
    optimizer.zero_grad(set_to_none=True)

    #computes gradients
    loss.backward()

    #updates model's weights based on gradients
    optimizer.step()

print(loss.item())

step: 0, train loss: 4.434, val loss: 4.431
step: 250, train loss: 1.804, val loss: 1.996
step: 500, train loss: 1.498, val loss: 1.764
step: 750, train loss: 1.351, val loss: 1.660
1.3516881465911865
